## Imports

In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.dummy import DummyClassifier

In [2]:
tqdm.pandas()

In [3]:
import sys
sys.path.append('../src/')
from models.classification_methods import process_classification 

## definitions

In [4]:
model_name = 'neuralmind/bert-base-portuguese-cased'

In [5]:
random_seed = 42

In [6]:
path_raw_data = '../data/raw/'
path_processed_data = '../data/processed/'
path_results_cr = '../reports/classification_report'

In [7]:
list_target = ['ig','bo', 'cl', 'co', 'gl', 'lu']

## Classification

### Top mentioned timelines

In [8]:
# create a list of tuples with (data_train, data_test, target)

list_tuples_top_ment = []

for target in tqdm(list_target):
    
    path_data_train = path_raw_data + f'train_r3_{target}_top_mentioned_timelines.csv'
    path_data_test = path_raw_data + f'test_r3_{target}_top_mentioned_timelines.csv'

    data_train = pd.read_csv(
        path_data_train, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    data_test = pd.read_csv(
        path_data_test, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    
    list_tuples_top_ment.append((data_train, data_test, target))

100%|██████████| 6/6 [00:22<00:00,  3.76s/it]


In [9]:
# get results

classifier = DummyClassifier()

df_cr, df_test_results = process_classification(
        estimator = classifier,
        data_tuples = list_tuples_top_ment
)

df_cr[df_cr['class'] == 'macro avg'].sort_values('f1-score')

Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Pro

/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,class,precision,recall,f1-score,support,corpus
3,macro avg,0.251742,0.5,0.334878,574.0,cl
3,macro avg,0.262868,0.5,0.344578,272.0,lu
3,macro avg,0.271318,0.5,0.351759,774.0,co
3,macro avg,0.282972,0.5,0.361407,599.0,ig
3,macro avg,0.296837,0.5,0.372519,411.0,gl
3,macro avg,0.430851,0.5,0.462857,188.0,bo


### Users

In [10]:
# create a list of tuples with (data_train, data_test, target)

list_tuples_users = []

for target in tqdm(list_target):
    
    path_data_train = path_raw_data + f'r3_{target}_train_users.csv'
    path_data_test = path_raw_data + f'r3_{target}_train_users.csv'

    data_train = pd.read_csv(
        path_data_train, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    data_test = pd.read_csv(
        path_data_test, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    
    list_tuples_users.append((data_train, data_test, target))

100%|██████████| 6/6 [00:54<00:00,  9.05s/it]


#### Timelines

In [11]:
# get results

classifier = DummyClassifier()

df_cr, df_test_results = process_classification(
        estimator = classifier,
        data_tuples = list_tuples_users,
        X_cols=['Timeline']
)

df_cr[df_cr['class'] == 'macro avg'].sort_values('f1-score')

Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Pro

/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) P

/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,class,precision,recall,f1-score,support,corpus
3,macro avg,0.251307,0.5,0.334493,1721.0,cl
3,macro avg,0.261642,0.5,0.343524,816.0,lu
3,macro avg,0.271022,0.5,0.351510,2319.0,co
3,macro avg,0.282572,0.5,0.361081,1796.0,ig
3,macro avg,0.296507,0.5,0.372259,1231.0,gl
3,macro avg,0.432504,0.5,0.463810,563.0,bo


#### Stance

In [12]:
# get results

classifier = DummyClassifier()

df_cr, df_test_results = process_classification(
        estimator = classifier,
        data_tuples = list_tuples_users,
        X_cols=['Stance']
)

df_cr[df_cr['class'] == 'macro avg'].sort_values('f1-score')

Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Processing selection, total=   0.0s
[Pipeline] ......... (step 5 of 5) Processing estimator, total=   0.0s
Training ...
[Pipeline] ........ (step 1 of 5) Processing vectorizer, total=   0.0s
[Pipeline] .......... (step 2 of 5) Processing sampling, total=   0.0s
[Pipeline] ........... (step 3 of 5) Processing scaling, total=   0.0s
[Pipeline] ......... (step 4 of 5) Pro

/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/semcovici/anaconda3/envs/env-stance-pred/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,class,precision,recall,f1-score,support,corpus
3,macro avg,0.251307,0.5,0.334493,1721.0,cl
3,macro avg,0.261642,0.5,0.343524,816.0,lu
3,macro avg,0.271022,0.5,0.351510,2319.0,co
3,macro avg,0.282572,0.5,0.361081,1796.0,ig
3,macro avg,0.296507,0.5,0.372259,1231.0,gl
3,macro avg,0.432504,0.5,0.463810,563.0,bo
